In [10]:
import os
from pathlib import Path
from typing import Optional

import evaluate
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from razdel import sentenize
from rouge_score import rouge_scorer

In [23]:
def find_repo_root(start: Optional[Path] | None = None) -> Optional[Path]:
    p = start or Path.cwd()
    for q in [p, *p.parents]:
        if (q / ".git").exists():
            return q
    return Path.cwd()  # запасной вариант


load_dotenv()

EXTERNAL = Path(os.getenv("EXTERNAL_STORAGE_DIR"))
SRC = EXTERNAL / "data" / "raw" / "gazeta_validation.jsonl"
ROOT = find_repo_root()
METRICS_DIR = ROOT / "metrics"
METRICS_DIR.mkdir(parents=True, exist_ok=True)

text_col, summ_col, title_col = "text", "reference_summary", "title"

In [9]:
df = pd.read_json(SRC, lines=True)

print(df.shape)
df.head(2)

(6369, 4)


,id,title,text,reference_summary
0,validation_0,Дорогой 2020-й: какие продукты подскочат в цене,"В 2020 году инфляция в России составит 3,5-4%,...",В уходящем году инфляция в России находится на...
1,validation_1,Подарок от Ким Чен Ына: Трамп ответил на новые...,Глава Белого дома Дональд Трамп выразил надежд...,Мировая общественность призвала лидера КНДР Ки...


In [11]:
df.loc[0, "reference_summary"]

'В уходящем году инфляция в России находится на историческом минимуме. В следующем году ожидается, что она также будет минимальной. Однако стоимость ряда продуктов и напитков в 2020 году может вырасти гораздо выше инфляции. Это касается молочных продуктов. Вырастет в цене водка, коньяк и вино, продолжит дорожать гречка.'

In [12]:
def lead_k(text: Optional[str], k: Optional[int]) -> Optional[str]:
    sents = [s.text.strip() for s in sentenize(text or "")]
    return " ".join(sents[:k])

In [13]:
preds = [lead_k(text, 3) for text in df[text_col]]
refs = df[summ_col].tolist()

In [14]:
preds[0]

'В 2020 году инфляция в России составит 3,5-4%, прогнозирует Центробанк. В первом квартале она ожидается так и вовсе ниже 3%. Впрочем, на ряд продовольственных товаров цены будут расти гораздо более высокими темпами.'

In [16]:
rouge = evaluate.load("rouge")
scores = rouge.compute(predictions=preds, references=refs, use_stemmer=False)
scores

{'rouge1': np.float64(0.22966416221096536),
 'rouge2': np.float64(0.08414394187762991),
 'rougeL': np.float64(0.22196393748599946),
 'rougeLsum': np.float64(0.2218676485892185)}

In [22]:
scores.get("rouge1", 0.0)

np.float64(0.22966416221096536)

In [24]:
Path(METRICS_DIR).mkdir(parents=True, exist_ok=True)

df_metrics = pd.DataFrame(
    [
        {
            "system": "extractive_lead3",
            "split": "validation_full",
            "rouge1": scores.get("rouge1", 0.0),
            "rouge2": scores.get("rouge2", 0.0),
            "rougeL": scores.get("rougeL", 0.0),
            "rougeLsum": scores.get("rougeLsum", 0.0),
            "avg_pred_len_tokens": np.mean([len(p.split()) for p in preds]),
            "k": 3,
            "n_examples": df.shape[0],
        }
    ]
)
df_metrics.to_csv(METRICS_DIR / "lead3_validation_full.csv", index=False)

df_sampels = pd.DataFrame(
    {
        "title": df["title"].head(3) if "title" in df else [""] * 3,
        "reference": refs[:3],
        "prediction": preds[:3],
    }
)
df_sampels.to_csv(METRICS_DIR / "lead3_validation_examples.tsv", sep="\t", index=False)
print("Save")

Save
